In [1]:
import pandas as pd
import numpy as np

# Chemins CSV (à modifier si besoin)
CSV_activity_RAW = "../data/activity-service/activity-service-data.csv"
CSV_activity_INTERPOLATED = "../data/activity-service/activity-service-data-interpolated.csv"
CSV_CRISE_COMPARAISON = "../data/activity-service/activity-service-data-with-crise.csv"

In [2]:
df = pd.read_csv(CSV_activity_RAW)
df.columns = df.columns.str.strip()

def to_number(x):
    if pd.isna(x):
        return np.nan
    x = str(x).replace("\u202f", "").replace(" ", "")
    x = x.replace(",", ".")
    try:
        return float(x)
    except:
        return np.nan

for col in ["PLF", "CFX", "TOTAL"]:
    df[col] = df[col].apply(to_number)

df = df.sort_values(["INDICATEUR", "SOUS-INDICATEUR", "ANNEE"])

def interpolate_group(g):
    indicateur, sous_indicateur = g.name

    # --- Unité (on assume une seule unité par couple indicateur / sous-indicateur)
    unite = g["UNITE"].dropna().iloc[0] if "UNITE" in g.columns and g["UNITE"].notna().any() else np.nan

    # --- Proportion PLF dans (PLF+CFX) quand on l'a (pour répartir TOTAL si besoin)
    ratio = np.nan
    mask_known = g["PLF"].notna() & g["CFX"].notna() & ((g["PLF"] + g["CFX"]) > 0)
    if mask_known.any():
        ratio = (g.loc[mask_known, "PLF"] / (g.loc[mask_known, "PLF"] + g.loc[mask_known, "CFX"])).median()
    if np.isnan(ratio):
        ratio = 0.5  # fallback

    g = g.set_index("ANNEE")
    year_min = min(2011, g.index.min())
    years = range(year_min, g.index.max() + 2)
    g = g.reindex(years)

    g["INDICATEUR"] = indicateur
    g["SOUS-INDICATEUR"] = sous_indicateur
    g["UNITE"] = unite

    # 1) Si TOTAL existe et CFX manquant -> CFX = TOTAL - PLF
    m = g["TOTAL"].notna() & g["PLF"].notna() & g["CFX"].isna()
    g.loc[m, "CFX"] = g.loc[m, "TOTAL"] - g.loc[m, "PLF"]

    # 2) Si TOTAL existe et PLF manquant -> PLF = TOTAL - CFX
    m = g["TOTAL"].notna() & g["CFX"].notna() & g["PLF"].isna()
    g.loc[m, "PLF"] = g.loc[m, "TOTAL"] - g.loc[m, "CFX"]

    # 3) Si TOTAL existe et PLF+CFX manquent -> répartir via ratio
    m = g["TOTAL"].notna() & g["PLF"].isna() & g["CFX"].isna()
    g.loc[m, "PLF"] = g.loc[m, "TOTAL"] * ratio
    g.loc[m, "CFX"] = g.loc[m, "TOTAL"] * (1 - ratio)

    # 4) Interpolation linéaire
    g["PLF"] = g["PLF"].interpolate(method="linear", limit_direction="both")
    g["CFX"] = g["CFX"].interpolate(method="linear", limit_direction="both")

    # 5) Compléter TOTAL si manquant
    mt = g["TOTAL"].isna()
    g.loc[mt, "TOTAL"] = g.loc[mt, "PLF"] + g.loc[mt, "CFX"]

    # 6) Arrondir toutes les données numériques à 2 décimales
    g[["PLF", "CFX", "TOTAL"]] = g[["PLF", "CFX", "TOTAL"]].round(2)

    return g.reset_index().rename(columns={"index": "ANNEE"})

df_interpolated = (
    df.groupby(["INDICATEUR", "SOUS-INDICATEUR"], group_keys=False)
      .apply(interpolate_group)
)

print("NaN restants :\n", df_interpolated[["PLF","CFX","TOTAL"]].isna().sum())
df_interpolated.to_csv(CSV_activity_INTERPOLATED, index=False)
print("Fichier sauvegardé ✅")

FileNotFoundError: [Errno 2] No such file or directory: '../data/activity-service/activity-service-data.csv'

In [3]:
df = pd.read_csv(CSV_activity_INTERPOLATED)

# --- 1) Dataset normal
df_normal = df.copy()
df_normal["MODE"] = "Normal"

# --- 2) Coefficient crise global : +70% pour tous
COEF_CRISE_GLOBAL = 1.70

# --- 3) Dataset crise
df_crise = df.copy()
df_crise["MODE"] = "Crise"
df_crise["COEF_CRISE"] = COEF_CRISE_GLOBAL

df_crise["PLF"] = (df_crise["PLF"] * df_crise["COEF_CRISE"]).round(2)
df_crise["CFX"] = (df_crise["CFX"] * df_crise["COEF_CRISE"]).round(2)
df_crise["TOTAL"] = (df_crise["PLF"] + df_crise["CFX"]).round(2)

# --- 4) Calcul évolution (Crise vs Normal)
merge_cols = ["ANNEE", "INDICATEUR", "SOUS-INDICATEUR"]

df_compare = df_normal.merge(
    df_crise[merge_cols + ["PLF", "CFX", "TOTAL"]],
    on=merge_cols,
    suffixes=("_NORMAL", "_CRISE")
)

# Écart en valeur et variation en %
df_compare["ECART_TOTAL"] = (df_compare["TOTAL_CRISE"] - df_compare["TOTAL_NORMAL"]).round(2)
df_compare["VARIATION_PCT"] = ((df_compare["ECART_TOTAL"] / df_compare["TOTAL_NORMAL"]) * 100).round(2)

print(df_compare.head(10))

# Sauvegarde
df_compare.to_csv(CSV_CRISE_COMPARAISON, index=False)
print("Fichier comparaison sauvegardé ✅")

FileNotFoundError: [Errno 2] No such file or directory: '../data/activity-service/activity-service-data-interpolated.csv'